## setup data path final norm

In [1]:
import numpy as np
import os
import pandas as pd
import pprint
import csv
import keras
#path constants
train_path = '../../../iterations/data/final/train_norm'
test_path = '../../../iterations/data/final/test_norm'

cleaned_train_path = '../../../cleaned_data/train'
cleaned_test_path = '../../../cleaned_data/test'
#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv('../../../iterations/data/final' + '/label.csv', delimiter = ',', encoding = 'utf-8')

Using TensorFlow backend.


## decide length of the input

In [2]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def cal_length(path, vehicle_type, label_df):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    single_len=0
    file_count=0
    file_len=0
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        file_len+=len(sample_df)
        file_count+=1
        if len(sample_df)>=single_len:
            single_len=len(sample_df)
    ave_file_len=file_len/file_count
    print("AVG Length: ",ave_file_len,' file count: ',file_count,' max_len: ',single_len)

In [3]:
cal_length(train_path,'ZV63d',train_label_df)

AVG Length:  167.2408891186353  file count:  3869  max_len:  720


## load the data

In [2]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def TraverseFiles(path, vehicle_type, label_df, length):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    lables=np.array([])
    file_list= []
    #file_len=file_len_weighted
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        if len(sample_df)==0:
            continue
        elif len(sample_df)!=0:
            sample_array=np.array(sample_df.iloc[:,0:11])
            if len(sample_array)<length:
                n_zeros=length-len(sample_array)
                sample_array=np.concatenate((np.zeros((n_zeros,11)),sample_array))
                #print(len(sample_array))
            elif len(sample_array)>length:
                sample_array=sample_array[0:length]
                #print(len(sample_array))
            elif len(sample_array)==length:
                sample_array=sample_array
                #print(len(sample_array))
            #np.float(sample_array)
            #sample_array=torch.from_numpy(sample_array).float()
            #file_list=torch.cat((file_list,sample_array[0:76,]), dim=1)
        file_list.append(sample_array)
        l=np.array(label_df.loc[label_df['sample_file_name'] == file])
        lables=np.append(lables,[l[:,1]])
        #belows are to show traversing progress (DO NOT CHANGE)
        counter += 1
        if counter == counts_per_percent:
            counter = 0
            percentage_completion += 1
            print('traversing files under', path, ':', percentage_completion, "%", end="\r", flush=True)

    return file_list,lables

In [5]:
data_array_train, data_labels_train = TraverseFiles(train_path,'ZV63d',train_label_df,170)
data_array_test, data_labels_test = TraverseFiles(test_path,'ZV63d',test_label_df,170)

In [6]:
x_train=np.array(data_array_train)
labels_train=np.array(data_labels_train,dtype=int)

In [7]:
x_test=np.array(data_array_test)
labels_test=np.array(data_labels_test,dtype=int)

In [8]:
print(x_train.shape)
print(labels_train.shape)
print(x_test.shape)
print(labels_test.shape)

(3869, 170, 11)
(3869,)
(968, 170, 11)
(968,)


## build and train model

In [9]:
keras.backend.clear_session()
lstm = keras.Sequential()
lstm.add(keras.layers.Dropout(0.2))
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.LSTM(16,dropout=0.2, recurrent_dropout=0.2))
lstm.add(keras.layers.BatchNormalization())
lstm.add(keras.layers.Dense(1, activation = 'sigmoid'))

lstm.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
)

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
lstm.fit(x_train,
    labels_train,
    epochs = 500,
    batch_size = 200
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
3869/3869 [==============================] - 4s 1ms/step - loss: 0.7924 - accuracy: 0.5030
Epoch 2/500
3869/3869 [==============================] - 3s 813us/step - loss: 0.7327 - accuracy: 0.5218
Epoch 3/500
3869/3869 [==============================] - 3s 819us/step - loss: 0.7132 - accuracy: 0.5255
Epoch 4/500
3869/3869 [==============================] - 3s 824us/step - loss: 0.7089 - accuracy: 0.5200
Epoch 5/500
3869/3869 [==============================] - 3s 831us/step - loss: 0.7043 - accuracy: 0.5304
Epoch 6/500
3869/3869 [==============================] - 3s 841us/step - loss: 0.6954 - accuracy: 0.5461
Epoch 7/500
3869/3869 [==============================] - 3s 860us/step - loss: 0.6977 - accuracy: 0.5428
Epoch 8/500
3869/3869 [==============================] - 3s 867us/step - loss: 0.6875 - accuracy: 0.5583
Epoch 9/500
3869/3869 [==============================] - 3s 869us/step - loss: 0.6927 - accuracy: 0.5510
Epoch 10/

3869/3869 [==============================] - 3s 814us/step - loss: 0.6545 - accuracy: 0.6304
Epoch 152/500
3869/3869 [==============================] - 3s 809us/step - loss: 0.6496 - accuracy: 0.6340
Epoch 153/500
3869/3869 [==============================] - 3s 806us/step - loss: 0.6569 - accuracy: 0.6286
Epoch 154/500
3869/3869 [==============================] - 3s 810us/step - loss: 0.6550 - accuracy: 0.6281
Epoch 155/500
3869/3869 [==============================] - 3s 810us/step - loss: 0.6558 - accuracy: 0.6252
Epoch 156/500
3869/3869 [==============================] - 3s 816us/step - loss: 0.6566 - accuracy: 0.6265
Epoch 157/500
3869/3869 [==============================] - 3s 807us/step - loss: 0.6559 - accuracy: 0.6263
Epoch 158/500
3869/3869 [==============================] - 3s 817us/step - loss: 0.6566 - accuracy: 0.6273
Epoch 159/500
3869/3869 [==============================] - 3s 814us/step - loss: 0.6551 - accuracy: 0.6317
Epoch 160/500
3869/3869 [==========================

3869/3869 [==============================] - 3s 838us/step - loss: 0.6487 - accuracy: 0.6374
Epoch 304/500
3869/3869 [==============================] - 3s 823us/step - loss: 0.6490 - accuracy: 0.6294
Epoch 305/500
3869/3869 [==============================] - 3s 828us/step - loss: 0.6495 - accuracy: 0.6319
Epoch 306/500
3869/3869 [==============================] - 3s 831us/step - loss: 0.6487 - accuracy: 0.6314
Epoch 307/500
3869/3869 [==============================] - 3s 825us/step - loss: 0.6441 - accuracy: 0.6423
Epoch 308/500
3869/3869 [==============================] - 3s 834us/step - loss: 0.6471 - accuracy: 0.6371
Epoch 309/500
3869/3869 [==============================] - 3s 820us/step - loss: 0.6499 - accuracy: 0.6350
Epoch 310/500
3869/3869 [==============================] - 3s 821us/step - loss: 0.6513 - accuracy: 0.6301
Epoch 311/500
3869/3869 [==============================] - 3s 871us/step - loss: 0.6458 - accuracy: 0.6384
Epoch 312/500
3869/3869 [==========================

3869/3869 [==============================] - 3s 842us/step - loss: 0.6408 - accuracy: 0.6400
Epoch 456/500
3869/3869 [==============================] - 3s 856us/step - loss: 0.6357 - accuracy: 0.6552
Epoch 457/500
3869/3869 [==============================] - 3s 837us/step - loss: 0.6413 - accuracy: 0.6428
Epoch 458/500
3869/3869 [==============================] - 3s 844us/step - loss: 0.6423 - accuracy: 0.6392
Epoch 459/500
3869/3869 [==============================] - 3s 822us/step - loss: 0.6456 - accuracy: 0.6376
Epoch 460/500
3869/3869 [==============================] - 3s 825us/step - loss: 0.6418 - accuracy: 0.6361
Epoch 461/500
3869/3869 [==============================] - 3s 824us/step - loss: 0.6444 - accuracy: 0.6358
Epoch 462/500
3869/3869 [==============================] - 3s 824us/step - loss: 0.6366 - accuracy: 0.6431
Epoch 463/500
3869/3869 [==============================] - 3s 827us/step - loss: 0.6397 - accuracy: 0.6449
Epoch 464/500
3869/3869 [==========================

In [13]:
lstm.save('lstm_model_ZV63d.h5')

## acc on the splitted test set

In [14]:
pred_=lstm.predict_classes(x_test)

In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(labels_test, pred_,digits=4))
print(accuracy_score(labels_test, pred_))

              precision    recall  f1-score   support

           0     0.6275    0.4442    0.5202       421
           1     0.6507    0.7971    0.7165       547

   micro avg     0.6436    0.6436    0.6436       968
   macro avg     0.6391    0.6206    0.6183       968
weighted avg     0.6406    0.6436    0.6311       968

0.643595041322314


## acc on cleaned test data

In [3]:
train_path = '../../../data/train'
test_path = '../../../data/test'

cleaned_train_path = '../../../cleaned_data/train'
cleaned_test_path = '../../../cleaned_data/test'
#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv(cleaned_train_path + '/train_label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv(cleaned_test_path + '/test_label.csv', delimiter = ',', encoding = 'utf-8')

In [5]:
cleaned_array_test, cleaned_labels_test = TraverseFiles(test_path,'ZV63d',test_label_df,170)

In [6]:
model = keras.models.load_model('lstm_model_ZV63d.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [7]:
cleaned_x=np.array(cleaned_array_test)
cleaned_labels=np.array(cleaned_labels_test,dtype=int)

In [8]:
pred_cleaned=model.predict_classes(cleaned_x)

In [9]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(classification_report(cleaned_labels, pred_cleaned,digits=4))
print(accuracy_score(cleaned_labels, pred_cleaned))

              precision    recall  f1-score   support

           0     0.3902    0.0166    0.0318       965
           1     0.5590    0.9796    0.7118      1228

   micro avg     0.5559    0.5559    0.5559      2193
   macro avg     0.4746    0.4981    0.3718      2193
weighted avg     0.4847    0.5559    0.4126      2193

0.555859553123575
